In [ ]:
!pip install pandas scikit-learn torch transformers imbalanced-learn nlpaug

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from torch.utils.data import Dataset
import nlpaug.augmenter.word as naw
from sklearn.metrics import classification_report
import csv

# 데이터 전처리

In [ ]:
# CSV 데이터 로드
df = pd.read_csv('final.csv', quoting=csv.QUOTE_NONE, on_bad_lines='skip')

In [ ]:
# 댓글 열을 문자열로 변환하고 결측값 제거
df['comments'] = df['comments'].astype(str)
df.dropna(subset=['comments', 'type'], inplace=True)

In [ ]:
# 데이터 8:2로 나누기
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# 원본 훈련 데이터 분포 확인
print("Original training set distribution:")
print(train_df['type'].value_counts())

In [ ]:
# 동의어 교체를 통한 데이터 증강
aug = naw.SynonymAug(aug_src='wordnet')

def augment_text(text):
    return aug.augment(text)

In [ ]:
# 부족한 클래스(3번) 데이터 증강
minority_class = train_df[train_df['type'] == 3]
augmented_texts = [augment_text(text) for text in minority_class['comments']]
augmented_df = pd.DataFrame({'comments': augmented_texts, 'type': [3] * len(augmented_texts)})

In [ ]:
# 원본 데이터와 증강 데이터를 합침
train_df_augmented = pd.concat([train_df, augmented_df])

In [ ]:
# 2번 클래스 추가 증강
augmented_texts = [augment_text(text) for text in train_df_augmented[train_df_augmented['type'] == 2]['comments']]
augmented_df = pd.DataFrame({'comments': augmented_texts, 'type': [2] * len(augmented_texts)})
train_df_augmented = pd.concat([train_df_augmented, augmented_df])

In [ ]:
# 증강된 훈련 데이터 분포 확인
print("Augmented training set distribution:")
print(train_df_augmented['type'].value_counts())

In [ ]:
# 0번 클래스와 4번 클래스의 데이터 수를 임의로 줄이기
class_0_count = 80000  # 0번 클래스 데이터 수
class_4_count = 40000  # 4번 클래스 데이터 수

class_0_data = train_df_augmented[train_df_augmented['type'] == 0].sample(class_0_count, random_state=42)
class_4_data = train_df_augmented[train_df_augmented['type'] == 4].sample(class_4_count, random_state=42)

In [ ]:
# 나머지 클래스 데이터 합침
other_classes_data = train_df_augmented[(train_df_augmented['type'] != 0) & (train_df_augmented['type'] != 4)]
train_df_balanced = pd.concat([class_0_data, class_4_data, other_classes_data])

In [ ]:
# 최종 훈련 데이터 분포 확인
print("Balanced training set distribution:")
print(train_df_balanced['type'].value_counts())

# 데이터셋 클래스 정의

In [ ]:
# 데이터셋 클래스 정의
class CommentDataset(Dataset):
    def __init__(self, comments, labels, tokenizer, max_length):
        self.comments = comments
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        comment = self.comments[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            comment,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

# 토크나이저 및 모델 로드

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

In [ ]:
# 훈련 데이터셋 생성
train_dataset = CommentDataset(
    comments=train_df_balanced['comments'].tolist(),
    labels=train_df_balanced['type'].tolist(),
    max_length=128
)

In [ ]:
# 테스트 데이터셋 생성
test_dataset = CommentDataset(
    comments=test_df['comments'].tolist(),
    labels=test_df['type'].tolist(),
    tokenizer=tokenizer,
    max_length=128
)

In [ ]:
# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels=len(df['type'].unique()))

# TrainingArguments 설정 및 Trainer 생성

In [ ]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='saved_model/type/results',
    num_train_epochs=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='saved_model/type/logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=True
)

In [ ]:
# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 모델 훈련

In [ ]:
# 모델 훈련
trainer.train()

# 모델 평가 및 성능 평가

In [ ]:
# 모델 평가
results = trainer.evaluate()
print(results)

In [ ]:
# 성능 평가
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids
print(classification_report(labels, preds))

# 모델 저장

In [ ]:
# 모델 저장
model.save_pretrained('saved_model/type')
tokenizer.save_pretrained('saved_model/type')

print("Model and tokenizer saved to 'saved_model/type'")